# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [14]:
import pandas as pd
import numpy as np
import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [7]:
df = pd.read_csv('previsao_de_renda.csv')
df.drop("Unnamed: 0",axis=1,inplace=True)
df.reset_index()

,level_0,data_ref,index,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,mau,renda
0,0,2015-01-01,8682,F,False,True,1,Assalariado,Secundário,Casado,Casa,36,3.575342,3.0,False,3369.24
1,1,2015-01-01,12830,M,True,True,0,Empresário,Secundário,Casado,Casa,42,0.860274,2.0,False,6096.14
2,2,2015-01-01,2885,M,True,True,2,Assalariado,Superior completo,Casado,Casa,31,8.065753,4.0,False,5658.98
3,3,2015-01-01,16168,F,True,False,0,Empresário,Secundário,Casado,Casa,50,1.208219,2.0,False,7246.69
4,4,2015-01-01,12438,M,False,False,0,Assalariado,Secundário,Casado,Casa,52,13.873973,2.0,False,4017.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,14995,2016-03-01,3678,M,True,True,1,Assalariado,Secundário,Casado,Casa,37,2.046575,3.0,False,1193.29
14996,14996,2016-03-01,3405,M,True,True,1,Servidor público,Superior completo,Casado,Estúdio,30,12.079452,3.0,False,2519.18
14997,14997,2016-03-01,7944,F,True,True,0,Assalariado,Secundário,Casado,Casa,40,2.161644,2.0,False,1586.85
14998,14998,2016-03-01,16019,F,False,True,0,Servidor público,Superior completo,Casado,Casa,47,3.295890,2.0,False,6975.39


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   data_ref               15000 non-null  object 
 1   index                  15000 non-null  int64  
 2   sexo                   15000 non-null  object 
 3   posse_de_veiculo       15000 non-null  bool   
 4   posse_de_imovel        15000 non-null  bool   
 5   qtd_filhos             15000 non-null  int64  
 6   tipo_renda             15000 non-null  object 
 7   educacao               15000 non-null  object 
 8   estado_civil           15000 non-null  object 
 9   tipo_residencia        15000 non-null  object 
 10  idade                  15000 non-null  int64  
 11  tempo_emprego          12466 non-null  float64
 12  qt_pessoas_residencia  15000 non-null  float64
 13  mau                    15000 non-null  bool   
 14  renda                  15000 non-null  float64
dtypes:

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [54]:
#1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
print(df['tipo_residencia'].value_counts())
y, X = patsy.dmatrices("np.log(renda) ~ C(sexo,Treatment(1)) + C(posse_de_veiculo,Treatment(1)) + posse_de_imovel + C(tipo_renda, Treatment(1)) + C(educacao, Treatment(3)) + C(estado_civil, Treatment(1)) + C(tipo_residencia, Treatment(0))", data=df)
sm.OLS(y,X).fit().summary()

tipo_residencia
Casa             13566
Com os pais        637
Governamental      443
Aluguel            184
Estúdio            107
Comunitário         63
Name: count, dtype: int64


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.083
Model:                            OLS   Adj. R-squared:                  0.081
Method:                 Least Squares   F-statistic:                     67.37
Date:                Tue, 12 Sep 2023   Prob (F-statistic):          2.54e-261
Time:                        17:40:02   Log-Likelihood:                -17243.
No. Observations:               15000   AIC:                         3.453e+04
Df Residuals:                   14979   BIC:                         3.469e+04
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
=====================================================================================================================
                                                        coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------
Intercept                                             8.3142      0.388     21.428      0.000       7.554       9.075
C(sexo, Treatment(1))[T.F]                           -0.4021      0.015    -27.325      0.000      -0.431      -0.373
C(posse_de_veiculo, Treatment(1))[T.False]            0.0240      0.014      1.709      0.088      -0.004       0.052
posse_de_imovel[T.True]                               0.0873      0.014      6.382      0.000       0.061       0.114
C(tipo_renda, Treatment(1))[T.Assalariado]           -0.0945      0.382     -0.247      0.805      -0.844       0.655
C(tipo_renda, Treatment(1))[T.Empresário]             0.0274      0.383      0.072      0.943      -0.723       0.777
C(tipo_renda, Treatment(1))[T.Pensionista]           -0.2284      0.383     -0.597      0.551      -0.979       0.522
C(tipo_renda, Treatment(1))[T.Servidor público]       0.1306      0.383      0.341      0.733      -0.620       0.881
C(educacao, Treatment(3))[T.Primário]                -0.2385      0.057     -4.218      0.000      -0.349      -0.128
C(educacao, Treatment(3))[T.Pós graduação]            0.3142      0.176      1.787      0.074      -0.030       0.659
C(educacao, Treatment(3))[T.Secundário]              -0.0805      0.014     -5.961      0.000      -0.107      -0.054
C(educacao, Treatment(3))[T.Superior incompleto]     -0.1621      0.033     -4.894      0.000      -0.227      -0.097
C(estado_civil, Treatment(1))[T.Casado]               0.0266      0.028      0.961      0.336      -0.028       0.081
C(estado_civil, Treatment(1))[T.Solteiro]             0.0152      0.032      0.472      0.637      -0.048       0.078
C(estado_civil, Treatment(1))[T.União]               -0.0253      0.035     -0.730      0.465      -0.093       0.043
C(estado_civil, Treatment(1))[T.Viúvo]                0.0426      0.040      1.062      0.288      -0.036       0.121
C(tipo_residencia, Treatment(0))[T.Casa]              0.1157      0.057      2.032      0.042       0.004       0.227
C(tipo_residencia, Treatment(0))[T.Com os pais]      -0.0809      0.064     -1.260      0.208      -0.207       0.045
C(tipo_residencia, Treatment(0))[T.Comunitário]       0.1003      0.112      0.897      0.370      -0.119       0.319
C(tipo_residencia, Treatment(0))[T.Estúdio]           0.0557      0.093      0.597      0.550      -0.127       0.239
C(tipo_residencia, Treatment(0))[T.Governamental]     0.0403      0.067      0.599      0.549      -0.092       0.172
==============================================================================
Omnibus:                       39.398   Durbin-Watson:                   2.005
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               41.528
Skew:         